# Fine-tuning with Low-Rank Adaptation (LoRA)

In [3]:
import pandas as pd
from peft import LoraConfig, TaskType, get_peft_model
import transformers
from transformers import BertForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

transformers.logging.set_verbosity_error()

## Dataset loading

In [16]:
train_data = (load_dataset('csv', data_files='../data/train.csv', split='train')
              .class_encode_column('is_humor')
              .rename_column('is_humor', 'labels'))

dev_data = (load_dataset('csv', data_files='../data/dev.csv', split='train')
            .class_encode_column('is_humor')
            .rename_column('is_humor', 'labels'))

test_data = (load_dataset('csv', data_files='../data/test.csv', split='train')
             .class_encode_column('is_humor')
             .rename_column('is_humor', 'labels'))

## Model setup

In [17]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

In [18]:
train_data = train_data.map(tokenize, batched=True)
dev_data = dev_data.map(tokenize, batched=True)
test_data = test_data.map(tokenize, batched=True)

Map:   0%|          | 0/6400 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [19]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

## LoRA Setup 

In [20]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=1,
    lora_alpha=1,
    lora_dropout=0.1,
    bias="none"
)

In [21]:
peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

trainable params: 38,402 || all params: 109,522,180 || trainable%: 0.035063217331868304


In [22]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    # accuracy = accuracy_score(labels, predictions)
    # precision = precision_score(labels, predictions, average='weighted')
    # recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    
    return {
        # 'accuracy': accuracy,
        # 'precision': precision,
        # 'recall': recall,
        'f1': f1
    }
    

## Training setup

In [23]:
training_arguments = TrainingArguments(
    output_dir='bert-lora-humor-detection',
    evaluation_strategy="epoch",
    num_train_epochs=2,
)

In [24]:
trainer = Trainer(
    model=peft_model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=dev_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [25]:
trainer.train()

{'loss': 0.6242, 'learning_rate': 3.4375e-05, 'epoch': 0.62}
{'eval_loss': 0.4745866358280182, 'eval_f1': 0.8221189839572193, 'eval_runtime': 43.1229, 'eval_samples_per_second': 18.552, 'eval_steps_per_second': 2.319, 'epoch': 1.0}
{'loss': 0.5111, 'learning_rate': 1.8750000000000002e-05, 'epoch': 1.25}
{'loss': 0.3836, 'learning_rate': 3.125e-06, 'epoch': 1.88}
{'eval_loss': 0.349575400352478, 'eval_f1': 0.8605523149638341, 'eval_runtime': 36.823, 'eval_samples_per_second': 21.726, 'eval_steps_per_second': 2.716, 'epoch': 2.0}
{'train_runtime': 1625.9213, 'train_samples_per_second': 7.872, 'train_steps_per_second': 0.984, 'train_loss': 0.4949262857437134, 'epoch': 2.0}


TrainOutput(global_step=1600, training_loss=0.4949262857437134, metrics={'train_runtime': 1625.9213, 'train_samples_per_second': 7.872, 'train_steps_per_second': 0.984, 'train_loss': 0.4949262857437134, 'epoch': 2.0})

In [26]:
trainer.predict(test_data).metrics

{'test_loss': 0.3577665686607361,
 'test_f1': 0.8537736197348548,
 'test_runtime': 37.9098,
 'test_samples_per_second': 21.103,
 'test_steps_per_second': 2.638}